## Utility Data Standardizer package
The goal of this package is to use python and pandas to create a package that can read different types of utility data files I've seen, and automatically convert them to a standardized format, with stanard column names, intervals, and visualizations that may be used to derive valuable insights.  

The first step will be to build a standardize_file function that can take in csvs and excel files, apply column mapping, normalizes datetime and units, infers energy type and year, resamples to monthly, and then exports to exel.  

Then, 15-min interval data analysis, aggregation, concatenation, and visualization

In [333]:
%pip install pyyaml


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [334]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import yaml
from dateutil import parser
from io import BytesIO
import xlsxwriter


In [335]:
load_column_mapping('column_mappings.yaml')

{'cost_usd': ['cost', 'charge', 'total $', 'total cost', '$', 'amount'],
 'demand_kW': ['demand', 'peak', 'max demand', 'kw', 'kw peak', 'peak demand'],
 'meter_id': ['meter',
  'meter number',
  'meter_id',
  'service_id',
  'account number'],
 'timestamp': ['date',
  'datetime',
  'interval start',
  'interval',
  'time',
  'timestamp'],
 'usage_kWh': ['usage',
  'energy use',
  'kwh',
  'kwh used',
  'consumption',
  'therms',
  'energy consumed']}

In [336]:

def data_to_df_reader(filepath, column_mapping):
    """
    Reads a CSV or Excel file and returns a standardized DataFrame
    using provided column mapping. Handles varied column names,
    timestamps, and adds a source_file column.

    Parameters:
    - filepath (str): path to the file to read
    - column_mapping (dict): maps standard column names to a list of known variations

    Returns:
    - pd.DataFrame: standardized DataFrame with renamed columns
    """

    # Determine file extension
    ext = os.path.splitext(filepath)[-1].lower()

    # Read file based on extension
    if ext == '.csv':
        df = pd.read_csv(filepath)
    elif ext in ['.xlsx', '.xls']:
        df = pd.read_excel(filepath, engine='openpyxl')
    else:
        raise ValueError(f"Unsupported file type: {ext}")
    
    
    # Build case-insensitive lookup that maps variations to standard names
    mapped_cols = {}

    # Additional logic to map columns based on substring matching
    for col in df.columns:
        col_lower = col.lower()
        if 'kwh' in col_lower or 'usage' in col_lower or 'consumption' in col_lower:
            mapped_cols[col] = 'usage_kWh'
        elif 'kw' in col_lower or 'demand' in col_lower or 'peak' in col_lower:
            mapped_cols[col] = 'demand_kW'
        elif 'time' in col_lower or 'date' in col_lower or 'interval' in col_lower:
            mapped_cols[col] = 'timestamp'
        elif '$' in col_lower or 'cost' in col_lower or 'amount' in col_lower:
            mapped_cols[col] = 'cost_usd'
        elif 'therm' in col_lower:
            mapped_cols[col] = 'usage_therms'
        elif 'meter' in col_lower or 'account' in col_lower or 'service id' in col_lower:
            mapped_cols[col] = 'meter_id'

    # Rename columns
    df = df.rename(columns=mapped_cols)

    # Retain only standardized columns that were matched
    df = df[list(mapped_cols.values())]
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    else:
        raise ValueError(f"'timestamp' column not found after mapping in file: {filepath}")




    # Add filename source
    df['source_file'] = os.path.basename(filepath)

    # Drop completely empty columns (optional)
    df.dropna(axis=1, how='all', inplace=True)

    return df


In [337]:
data_to_df_reader('test_utility_data.xlsx', load_column_mapping('column_mappings.yaml'))

,timestamp,usage_kWh,demand_kW,meter_id,cost_usd,source_file
0,2024-01-01 00:00:00,0.25,1.2,12345,0.030,test_utility_data.xlsx
1,2024-01-01 00:15:00,0.30,1.4,12345,0.035,test_utility_data.xlsx
2,2024-01-01 00:30:00,0.35,1.6,12345,0.040,test_utility_data.xlsx
3,2024-01-01 00:45:00,0.40,1.8,12345,0.045,test_utility_data.xlsx
4,2024-01-01 01:00:00,0.45,2.0,12345,0.050,test_utility_data.xlsx
5,2024-01-01 01:15:00,0.50,1.2,12345,0.030,test_utility_data.xlsx
6,2024-01-01 01:30:00,0.55,1.4,12345,0.035,test_utility_data.xlsx
7,2024-01-01 01:45:00,0.60,1.6,12345,0.040,test_utility_data.xlsx
8,2024-01-01 02:00:00,0.65,1.8,12345,0.045,test_utility_data.xlsx
9,2024-01-01 02:15:00,0.70,2.0,12345,0.050,test_utility_data.xlsx


In [338]:
import zipfile

#additional functrion that concatenates monthly interval data into a single DataFrame
def concatenate_monthly_data(input_path, column_mapping):
    """
    Concatenates multiple monthly utility data files from a zip file or folder into a single DataFrame.

    Parameters:
    - input_path (str): Path to the zip file or folder containing the monthly data files.
    - column_mapping (dict): Maps standard column names to a list of known variations.

    Returns:
    - pd.DataFrame: Concatenated DataFrame with standardized columns.
    """

    dfs = []
    filepaths = []

    # Check if input_path is a zip file
    if zipfile.is_zipfile(input_path):
        with zipfile.ZipFile(input_path, 'r') as zip_ref:
            # Extract all files to a temporary directory
            temp_dir = 'temp_extracted_files'
            zip_ref.extractall(temp_dir)
            # Get all file paths from the extracted folder
            filepaths = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith(('.csv', '.xlsx'))]
    else:
        # Assume input_path is a folder
        filepaths = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith(('.csv', '.xlsx'))]

    # Process each file
    for filepath in filepaths:
        df = data_to_df_reader(filepath, column_mapping)
        dfs.append(df)
    
    # Concatenate all DataFrames
    combined_df = pd.concat(dfs, ignore_index=True).sort_values(by='timestamp')
    
    return combined_df

In [339]:
#next step is to create a function that uses the created dataframe to show the monthly usage with visualizations
def utility_df_aggregator(df):
    """
    Aggregates utility data by month and visualizes the results.

    Parameters:
    - df (pd.DataFrame): DataFrame containing utility data with standardized columns

    Returns:
    - pd.DataFrame: Aggregated monthly usage DataFrame
    """
    
    # Ensure timestamp is in datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # Set timestamp as index
    df.set_index('timestamp', inplace=True)

    # Group by month and aggregate usage
    monthly_usage = df.resample('M').sum()['usage_kWh']

    monthly_peak_demand = df.resample('M').max()['demand_kW']

    # Extract the year from the first timestamp for the title
    year = monthly_usage.index[0].year

    # Setup figure and axes
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Bar chart for usage
    bars = ax1.bar(monthly_usage.index.month, monthly_usage.values, color='blue', label='Usage (kWh)', width=0.8)
    ax1.set_ylabel('Usage (kWh)')
    ax1.set_xlabel('Month')
    ax1.set_xticks(range(1, 13))
    ax1.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

    # Line chart for peak demand
    ax2 = ax1.twinx()
    ax2.plot(monthly_peak_demand.index.month, monthly_peak_demand.values, color='orange', marker='o', label='Peak Demand (kW)')
    ax2.set_ylim(0, monthly_peak_demand.max() * 1.1)  # Set y-limits for better visibility
    ax2.set_ylabel('Peak Demand (kW)')

    # Legends
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    plt.title(f'Monthly Electricity Usage and Peak Demand ({year})')
    plt.tight_layout()
    chart_img = BytesIO()
    plt.savefig(chart_img, format='png')
    plt.close(fig)  # Close the figure to free memory
    # Return the aggregated DataFrame and chart image
    
    return monthly_usage, monthly_peak_demand, chart_img

        

In [340]:
# Load column mapping
column_mapping = load_column_mapping('column_mappings.yaml')

# Concatenate monthly data
combined_df = concatenate_monthly_data('/Users/aldenblackman/Downloads/Utility Data Standardizer Project/synthetic_monthly_files.zip', column_mapping)

combined_df


,timestamp,usage_kWh,demand_kW,meter_id,cost_usd,source_file
11520,2024-01-01 00:00:00,0.820,3.75,MTR123,0.2675,synthetic_2024_01.xlsx
11521,2024-01-01 00:15:00,1.422,1.69,MTR123,0.2539,synthetic_2024_01.xlsx
11522,2024-01-01 00:30:00,1.150,3.01,MTR123,0.0840,synthetic_2024_01.xlsx
11523,2024-01-01 00:45:00,0.978,2.73,MTR123,0.2859,synthetic_2024_01.xlsx
11524,2024-01-01 01:00:00,0.871,3.58,MTR123,0.2701,synthetic_2024_01.xlsx
...,...,...,...,...,...,...
23035,2024-12-30 22:45:00,1.195,2.67,MTR123,0.0574,synthetic_2024_12.xlsx
23036,2024-12-30 23:00:00,0.160,4.10,MTR123,0.2070,synthetic_2024_12.xlsx
23037,2024-12-30 23:15:00,1.029,4.13,MTR123,0.1354,synthetic_2024_12.xlsx
23038,2024-12-30 23:30:00,0.746,2.28,MTR123,0.0887,synthetic_2024_12.xlsx


In [341]:


# Aggregate and visualize the utility data
utility_df_aggregator(combined_df)





/var/folders/5x/4m5cfl9j4x1gc07kmkljv6tr0000gn/T/ipykernel_23564/3295804738.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_usage = df.resample('M').sum()['usage_kWh']
/var/folders/5x/4m5cfl9j4x1gc07kmkljv6tr0000gn/T/ipykernel_23564/3295804738.py:22: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_peak_demand = df.resample('M').max()['demand_kW']


(timestamp
 2024-01-31    2337.123
 2024-02-29    2207.364
 2024-03-31    2391.985
 2024-04-30    2283.843
 2024-05-31    2302.923
 2024-06-30    2277.492
 2024-07-31    2279.427
 2024-08-31    2343.871
 2024-09-30    2279.692
 2024-10-31    2307.546
 2024-11-30    2328.949
 2024-12-31    2299.651
 Freq: ME, Name: usage_kWh, dtype: float64,
 timestamp
 2024-01-31    5.00
 2024-02-29    5.00
 2024-03-31    5.00
 2024-04-30    5.00
 2024-05-31    5.00
 2024-06-30    5.00
 2024-07-31    5.00
 2024-08-31    5.00
 2024-09-30    5.00
 2024-10-31    4.99
 2024-11-30    5.00
 2024-12-31    5.00
 Freq: ME, Name: demand_kW, dtype: float64,
 <_io.BytesIO at 0x16a021d50>)

In [342]:
%pip install xlsxwriter

def export_summary_to_excel(monthly_usage, monthly_peak_demand, chart_img, output_file):
    """
    Exports the monthly usage and peak demand data along with a chart image to an Excel file.

    Parameters:
    - monthly_usage (pd.Series): Monthly usage data.
    - monthly_peak_demand (pd.Series): Monthly peak demand data.
    - chart_img (BytesIO): In-memory image of the chart.
    - output_file (str): Path to the output Excel file.
    """
    
    # Create a Pandas Excel writer using XlsxWriter as the engine
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        # Write monthly usage and peak demand to separate sheets
        monthly_usage.to_frame(name='Usage (kWh)').to_excel(writer, sheet_name='Monthly Usage')
        monthly_peak_demand.to_frame(name='Peak Demand (kW)').to_excel(writer, sheet_name='Monthly Peak Demand')

        # Add a chart image to the workbook
        workbook = writer.book
        worksheet = workbook.add_worksheet('Charts')
        
        # Insert the chart image
        worksheet.insert_image('A1', 'chart.png', {'image_data': chart_img})

    print(f"Summary exported to {output_file}")



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [343]:
#final example workflow
# Ensure the DataFrame has the 'timestamp' column as a regular column
combined_df.reset_index(inplace=True)

# Aggregate and visualize the utility data
monthly_usage, monthly_peak, chart_img = utility_df_aggregator(combined_df)

# Export the summary to Excel
export_summary_to_excel(monthly_usage, monthly_peak, chart_img, "MyFacility_2024_Electric_STANDARDIZED.xlsx")


Summary exported to MyFacility_2024_Electric_STANDARDIZED.xlsx


/var/folders/5x/4m5cfl9j4x1gc07kmkljv6tr0000gn/T/ipykernel_23564/3295804738.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_usage = df.resample('M').sum()['usage_kWh']
/var/folders/5x/4m5cfl9j4x1gc07kmkljv6tr0000gn/T/ipykernel_23564/3295804738.py:22: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_peak_demand = df.resample('M').max()['demand_kW']
